<h1 align=center><font size = 6>The Battle of Neighborhoods (Week 1)</font></h1>

<h1 align=center><font size = 5>Su Yiping</font></h1>

## Introduction

In this project, I will analysis of various venues for finding useful insights into the kind of business thriving in difference locality. Analysis will be based on the number and category of venues of various types present in Singapore.And help a group of stakeholders solve a problem about acquisition and sell of real estate.

- Scrape the following Wikipedia page, https://en.wikipedia.org/wiki/Postal_codes_in_Singapore, for getting the data that is in the table of postal codes and to transform the data
- Get the geographical coordinates of the neighborhoods in the Toronto.
- Generate maps to visualize your neighborhoods and how they cluster together

In [1]:
# importing necessary libraries
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
import geocoder
import folium   # map rendering library
from geopy.geocoders import Nominatim  # OSM(OpenStreetMap) data
from sklearn.cluster import KMeans
from tqdm import tqdm
from collections import deque
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np

In [2]:
url = "https://en.wikipedia.org/wiki/Postal_codes_in_Singapore"
wiki_page = requests.get(url).text

# using beautiful soup to parse the HTML/XML codes.
soup = BeautifulSoup(wiki_page, 'html.parser')

In [3]:
# Cleanup Data and Convert content of Postal district HTML table as list of data
data = []
for tr in soup.tbody.find_all('tr'):
    data.append([ td.get_text().strip() for td in tr.find_all('td')])

In [4]:
#Create the dataframe
df = pd.DataFrame(data, columns=['Postal district','Postal sector','General location'])
df.dropna(inplace=True) #Remove all "None" rows
df.head()

,Postal district,Postal sector,General location
1,01,"01, 02, 03, 04, 05, 06","Raffles Place, Cecil, Marina, People's Park"
2,02,"07, 08","Anson, Tanjong Pagar"
3,03,"14, 15, 16","Bukit Merah, Queenstown, Tiong Bahru"
4,04,"09, 10","Telok Blangah, Harbourfront"
5,05,"11, 12, 13","Pasir Panjang, Hong Leong Garden, Clementi New..."


In [5]:
# size of dataframe
df.shape

(28, 3)

In [6]:
# Add latitude and longitude to dataframe from district.json
district_data = None
with open('district.json') as json_data:
    district_data = json.load(json_data)
district_dataframe_Latitude =[]
district_dataframe_Longitude =[]
for data_district in district_data:
    district_dataframe_Latitude.append(data_district['latitude'])
    district_dataframe_Longitude.append(data_district['longitude'])
df["latitude"] = district_dataframe_Latitude
df["longitude"] = district_dataframe_Longitude
df

,Postal district,Postal sector,General location,latitude,longitude
1,01,"01, 02, 03, 04, 05, 06","Raffles Place, Cecil, Marina, People's Park",1.281632,103.857248
2,02,"07, 08","Anson, Tanjong Pagar",1.272167,103.842780
3,03,"14, 15, 16","Bukit Merah, Queenstown, Tiong Bahru",1.286189,103.817772
4,04,"09, 10","Telok Blangah, Harbourfront",1.271282,103.817022
5,05,"11, 12, 13","Pasir Panjang, Hong Leong Garden, Clementi New...",1.295909,103.777698
6,06,17,"High Street, Beach Road (part)",1.292577,103.849194
7,07,"18, 19","Middle Road, Golden Mile",1.300595,103.858292
8,08,"20, 21","Little India, Farrer Park, Jalan Besar, Lavender",1.310853,103.854434
9,09,"22, 23","Orchard, Cairnhill, River Valley",1.302264,103.839234
10,10,"24, 25, 26, 27","Ardmore, Bukit Timah, Holland Road, Tanglin",1.313139,103.806978


In [7]:
# extracted latitude and longitude of Singapore

address = 'Singapore'

geolocator = Nominatim() # get OSM(OpenStreetMap) data
location = geolocator.geocode(address) # get the geogragraphical coordinates of Toronto
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Singaproe are {}, {}.'.format(latitude, longitude))

C:\Users\GUOCHENGGUANG\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """


The geograpical coordinate of Singaproe are 1.357107, 103.8194992.


In [8]:
# creat the Singapore map
map_Singapore = folium.Map(location = [latitude, longitude], zoom_start = 12)

# add markers to map
for lat, lng, location in zip(df['latitude'], df['longitude'], df['General location']):
    label = '{}'.format(location)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_Singapore)  

map_Singapore

In [9]:
#Configure Foursquare access
CLIENT_ID = 'TCJK5SCFXOAOICUOCPMYEQ1TMHFQ3QUTLJPHGXDYQHJFMDMX' # your Foursquare ID
CLIENT_SECRET = 'QUW4QNIGHKPXFQ5U3QC1LLYNNUCGLVRPCSID3IWYTFR25X40' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TCJK5SCFXOAOICUOCPMYEQ1TMHFQ3QUTLJPHGXDYQHJFMDMX
CLIENT_SECRET:QUW4QNIGHKPXFQ5U3QC1LLYNNUCGLVRPCSID3IWYTFR25X40


In [10]:
#Make a function that takes the names and locations of the location in Singapore
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT = 100):
    
    venues_list=[]
    for name, lat, lng in tqdm(zip(names, latitudes, longitudes), total = names.size):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Location', 
                  'Location Latitude', 
                  'Location Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [11]:
# obtains the 100 top venues around
singapore_venues = getNearbyVenues(df['General location'], df['latitude'], df['longitude'])

100%|██████████████████████████████████████████████████████████████████████████████████| 28/28 [00:23<00:00,  1.19it/s]


In [12]:
singapore_venues

,Location,Location Latitude,Location Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Raffles Place, Cecil, Marina, People's Park",1.281632,103.857248,Marina Bay Waterfront Promenade,1.281125,103.855556,Waterfront
1,"Raffles Place, Cecil, Marina, People's Park",1.281632,103.857248,New Creation Church (MBS),1.282423,103.858840,Church
2,"Raffles Place, Cecil, Marina, People's Park",1.281632,103.857248,Marina Bay Sands,1.283190,103.859124,Resort
3,"Raffles Place, Cecil, Marina, People's Park",1.281632,103.857248,Marina Bay Sands (Dock),1.281860,103.856978,Harbor / Marina
4,"Raffles Place, Cecil, Marina, People's Park",1.281632,103.857248,Din Tai Fung 鼎泰豐,1.282270,103.857608,Dumpling Restaurant
...,...,...,...,...,...,...,...
722,Seletar,1.407048,103.868455,Playground @ The Oval,1.409152,103.867402,Playground
723,Seletar,1.407048,103.868455,Airbus Helicopters Southeast Asia Pte Ltd,1.410629,103.868100,Office
724,Seletar,1.407048,103.868455,Orientus Resort Singapore,1.407786,103.871988,Resort
725,Seletar,1.407048,103.868455,Bali Culture,1.407849,103.872158,Gastropub


In [13]:
# the total amount of unique categories in my data
len(singapore_venues['Venue Category'].unique())

193

## Analyze Each Location

In [14]:
# take the venue category information and create a dataframe with a one hot enconding of these data
singapore_onehot = pd.get_dummies(singapore_venues["Venue Category"],
                             prefix = "",
                             prefix_sep = "")

singapore_onehot["Location"] = singapore_venues["Location"]


nindex = list(singapore_onehot.columns).index("Location")
cols = deque(singapore_onehot.columns)
cols.rotate(-nindex)
cols = list(cols)
singapore_onehot = singapore_onehot[cols]

singapore_onehot.head()

,Location,Accessories Store,Airport Terminal,American Restaurant,Art Gallery,Asian Restaurant,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,...,Train Station,Travel Lounge,Tunnel,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Women's Store,Yoga Studio
0,"Raffles Place, Cecil, Marina, People's Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,"Raffles Place, Cecil, Marina, People's Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Raffles Place, Cecil, Marina, People's Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Raffles Place, Cecil, Marina, People's Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Raffles Place, Cecil, Marina, People's Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
singapore_onehot.shape

(727, 194)

In [17]:
#compute the average number of venue categories per location
singapore_grouped = singapore_onehot.groupby('Location').mean().reset_index()
singapore_grouped

,Location,Accessories Store,Airport Terminal,American Restaurant,Art Gallery,Asian Restaurant,Australian Restaurant,BBQ Joint,Bagel Shop,Bakery,...,Train Station,Travel Lounge,Tunnel,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Women's Store,Yoga Studio
0,"Anson, Tanjong Pagar",0.000000,0.00,0.000000,0.000000,0.014925,0.000000,0.014925,0.014925,0.044776,...,0.000000,0.000000,0.00,0.000000,0.014925,0.000000,0.000000,0.000000,0.00,0.000000
1,"Ardmore, Bukit Timah, Holland Road, Tanglin",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,"Balestier, Toa Payoh, Serangoon",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
3,"Bedok, Upper East Coast, Eastwood, Kew Drive",0.000000,0.00,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,...,0.100000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
4,"Bishan, Ang Mo Kio",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
5,"Bukit Merah, Queenstown, Tiong Bahru",0.000000,0.00,0.000000,0.000000,0.034483,0.000000,0.000000,0.000000,0.034483,...,0.034483,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
6,"Geylang, Eunos",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
7,"High Street, Beach Road (part)",0.000000,0.00,0.000000,0.028986,0.014493,0.000000,0.000000,0.000000,0.043478,...,0.000000,0.000000,0.00,0.014493,0.000000,0.014493,0.014493,0.000000,0.00,0.014493
8,"Hillview, Dairy Farm, Bukit Panjang, Choa Chu ...",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,"Jurong, Tuas",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000


In [18]:
#Make a function that is the N most frequent venues 
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [19]:
#  Create the new dataframe and display the top 10 place for each location
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Location']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Place'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Place'.format(ind+1))

# create a new dataframe
Location_venues_sorted = pd.DataFrame(columns=columns)
Location_venues_sorted['Location'] = singapore_grouped['Location']

for ind in np.arange(singapore_grouped.shape[0]):
    Location_venues_sorted.iloc[ind, 1:] = return_most_common_venues(singapore_grouped.iloc[ind, :], num_top_venues)

Location_venues_sorted

,Location,1st Most Common Place,2nd Most Common Place,3rd Most Common Place,4th Most Common Place,5th Most Common Place,6th Most Common Place,7th Most Common Place,8th Most Common Place,9th Most Common Place,10th Most Common Place
0,"Anson, Tanjong Pagar",Japanese Restaurant,Coffee Shop,Café,Bakery,Food Court,Hotel,Soba Restaurant,Indian Restaurant,Fast Food Restaurant,Salad Place
1,"Ardmore, Bukit Timah, Holland Road, Tanglin",Bus Station,Noodle House,Metro Station,Bakery,Food Court,Yoga Studio,Flower Shop,Fujian Restaurant,Fruit & Vegetable Store,Frozen Yogurt Shop
2,"Balestier, Toa Payoh, Serangoon",Chinese Restaurant,Playground,Coffee Shop,Bus Station,Bridge,Grocery Store,Gym,Fruit & Vegetable Store,Tennis Court,Snack Place
3,"Bedok, Upper East Coast, Eastwood, Kew Drive",Playground,Train Station,Bus Station,School,Flower Shop,Noodle House,Basketball Court,Resort,Bus Line,Asian Restaurant
4,"Bishan, Ang Mo Kio",Chinese Restaurant,Grocery Store,Playground,Pharmacy,Food Court,Bubble Tea Shop,Park,Café,Restaurant,Health & Beauty Service
5,"Bukit Merah, Queenstown, Tiong Bahru",Chinese Restaurant,Coffee Shop,Fast Food Restaurant,Food Court,Gym / Fitness Center,Supermarket,Malay Restaurant,Hainan Restaurant,Sandwich Place,Residential Building (Apartment / Condo)
6,"Geylang, Eunos",Coffee Shop,Martial Arts Dojo,Motorcycle Shop,Grocery Store,Chinese Restaurant,Furniture / Home Store,Diner,Dumpling Restaurant,Food Court,Fujian Restaurant
7,"High Street, Beach Road (part)",Japanese Restaurant,Café,Shopping Mall,Supermarket,French Restaurant,Bakery,Coffee Shop,Art Gallery,Bistro,History Museum
8,"Hillview, Dairy Farm, Bukit Panjang, Choa Chu ...",Coffee Shop,Racetrack,Gaming Cafe,Fast Food Restaurant,Fujian Restaurant,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Court
9,"Jurong, Tuas",Military Base,Hostel,Outdoors & Recreation,Yoga Studio,Filipino Restaurant,Fujian Restaurant,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant


# Location clustering

Run *k*-means to cluster the location into 5 clusters.

In [20]:
# set number of clusters
kclusters = 5

singapore_grouped_clustering = singapore_grouped.drop('Location', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit( singapore_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_[0:10])
print(kmeans.labels_.shape)

[2 2 2 2 2 2 0 2 0 1]
(28,)


Create a dataframe that containes the location,coordinate and the cluster information, together with the top 10 venues

In [32]:
singapore_grouped["Cluster Labels"] = kmeans.labels_

# add clustering labels
singapore_combined = df.merge(singapore_grouped, left_on = "General location", right_on = "Location", how = "outer")

# merge singapore_grouped with toronto_data to add latitude/longitude for each Location
singapore_combined = singapore_combined.join(Location_venues_sorted.set_index('Location'), on='Location')

singapore_combined["Cluster Labels"] = singapore_combined["Cluster Labels"].fillna(5).astype("int")

singapore_combined.drop(['Location'], axis=1, inplace = True)
singapore_combined.head() # check the last columns!

,Postal district,Postal sector,General location,latitude,longitude,Accessories Store,Airport Terminal,American Restaurant,Art Gallery,Asian Restaurant,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,01,"01, 02, 03, 04, 05, 06","Raffles Place, Cecil, Marina, People's Park",1.281632,103.857248,0.021505,0.0,0.0,0.010753,0.000000,...,Boutique,Japanese Restaurant,Coffee Shop,Hotel,Waterfront,Theater,Italian Restaurant,Mexican Restaurant,Lounge,Jewelry Store
1,02,"07, 08","Anson, Tanjong Pagar",1.272167,103.842780,0.000000,0.0,0.0,0.000000,0.014925,...,Japanese Restaurant,Coffee Shop,Café,Bakery,Food Court,Hotel,Soba Restaurant,Indian Restaurant,Fast Food Restaurant,Salad Place
2,03,"14, 15, 16","Bukit Merah, Queenstown, Tiong Bahru",1.286189,103.817772,0.000000,0.0,0.0,0.000000,0.034483,...,Chinese Restaurant,Coffee Shop,Fast Food Restaurant,Food Court,Gym / Fitness Center,Supermarket,Malay Restaurant,Hainan Restaurant,Sandwich Place,Residential Building (Apartment / Condo)
3,04,"09, 10","Telok Blangah, Harbourfront",1.271282,103.817022,0.000000,0.0,0.0,0.000000,0.062500,...,Convenience Store,Café,Hotel,Mountain,Cable Car,Gym,Scenic Lookout,Board Shop,Park,Bus Line
4,05,"11, 12, 13","Pasir Panjang, Hong Leong Garden, Clementi New...",1.295909,103.777698,0.000000,0.0,0.0,0.000000,0.125000,...,Bus Station,Asian Restaurant,College Cafeteria,Coffee Shop,Bar,Salad Place,Gym / Fitness Center,Other Great Outdoors,Yoga Studio,Flower Shop


In [34]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

kclusters = kclusters + 1

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(singapore_combined['latitude'],
                                  singapore_combined['longitude'],
                                  singapore_combined['General location'],
                                  singapore_combined['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Analyze clusters

In [35]:
#cluster 1
singapore_combined.loc[singapore_combined['Cluster Labels'] == 0, "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Coffee Shop,Martial Arts Dojo,Motorcycle Shop,Grocery Store,Chinese Restaurant,Furniture / Home Store,Diner,Dumpling Restaurant,Food Court,Fujian Restaurant
22,Coffee Shop,Racetrack,Gaming Cafe,Fast Food Restaurant,Fujian Restaurant,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Court
26,Coffee Shop,Fast Food Restaurant,Basketball Court,Indian Restaurant,Ice Cream Shop,Asian Restaurant,Vegetarian / Vegan Restaurant,Thai Restaurant,Café,Flower Shop


In [36]:
#cluster 2
singapore_combined.loc[singapore_combined['Cluster Labels'] == 1, "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Military Base,Hostel,Outdoors & Recreation,Yoga Studio,Filipino Restaurant,Fujian Restaurant,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant


In [37]:
#cluster 3
singapore_combined.loc[singapore_combined['Cluster Labels'] == 2, "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Boutique,Japanese Restaurant,Coffee Shop,Hotel,Waterfront,Theater,Italian Restaurant,Mexican Restaurant,Lounge,Jewelry Store
1,Japanese Restaurant,Coffee Shop,Café,Bakery,Food Court,Hotel,Soba Restaurant,Indian Restaurant,Fast Food Restaurant,Salad Place
2,Chinese Restaurant,Coffee Shop,Fast Food Restaurant,Food Court,Gym / Fitness Center,Supermarket,Malay Restaurant,Hainan Restaurant,Sandwich Place,Residential Building (Apartment / Condo)
3,Convenience Store,Café,Hotel,Mountain,Cable Car,Gym,Scenic Lookout,Board Shop,Park,Bus Line
4,Bus Station,Asian Restaurant,College Cafeteria,Coffee Shop,Bar,Salad Place,Gym / Fitness Center,Other Great Outdoors,Yoga Studio,Flower Shop


In [38]:
#cluster 4
singapore_combined.loc[singapore_combined['Cluster Labels'] == 3, "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,Trail,Yoga Studio,Fast Food Restaurant,Furniture / Home Store,Fujian Restaurant,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Food Court


In [39]:
#cluster 5
singapore_combined.loc[singapore_combined['Cluster Labels'] == 4, "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Airport Terminal,Jewelry Store,Bus Stop,Skate Park,Yoga Studio,Filipino Restaurant,Fujian Restaurant,Fruit & Vegetable Store,Frozen Yogurt Shop,Fried Chicken Joint
